In [ ]:
"""
================================================================================
STUDENT TEST SCORE PREDICTION - COMPREHENSIVE TUTORIAL
================================================================================

This tutorial will guide you through building a machine learning model to predict
student test scores. We'll go step-by-step with theory and code together.

Author: Nikhil Chaudhary
Purpose: Educational - Learn ML by doing
Competition: Kaggle Playground Series S6E1
================================================================================
"""

# ============================================================================
# PART 1: SETUP AND IMPORTS
# ============================================================================

"""
THEORY - Why do we need these libraries?
=========================================

1. pandas (pd): For working with tabular data (like Excel spreadsheets)
   - Think of it as Excel in Python
   - We can filter, sort, calculate, and manipulate data easily

2. numpy (np): For numerical operations and arrays
   - Fast mathematical computations
   - Works well with pandas for calculations

3. matplotlib & seaborn: For creating visualizations
   - matplotlib: Low-level plotting (more control)
   - seaborn: High-level plotting (prettier, easier)

4. sklearn (scikit-learn): The main machine learning library
   - Contains algorithms, preprocessing tools, and evaluation metrics
   
5. warnings: To suppress unnecessary warning messages
"""

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')  # Makes plots look nice
sns.set_palette("husl")  # Color palette for plots

# Display settings for pandas
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show up to 100 rows
pd.set_option('display.float_format', '{:.2f}'.format)  # 2 decimal places

print("✓ All libraries imported successfully!")
print("=" * 80)


# ============================================================================
# PART 2: LOADING AND UNDERSTANDING THE DATA
# ============================================================================

"""
THEORY - What is our data?
===========================

In machine learning, we work with:
1. TRAINING DATA: Data we use to teach our model (has both features and target)
2. TESTING DATA: Data we use to make predictions (has features, no target)

FEATURES (X): The input variables we use to make predictions
   Examples: study hours, attendance, previous scores, etc.

TARGET (y): What we want to predict
   In this case: student test scores

The goal: Learn patterns from training data to predict scores for test data
"""

# Load the datasets
print("\n📂 LOADING DATA...")
print("-" * 80)

# Replace these paths with your actual file paths
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"✓ Training data loaded: {train_df.shape[0]} rows, {train_df.shape[1]} columns")
print(f"✓ Testing data loaded: {test_df.shape[0]} rows, {test_df.shape[1]} columns")
print("=" * 80)


# ============================================================================
# PART 3: INITIAL DATA EXPLORATION (First Look)
# ============================================================================

"""
THEORY - Why explore data first?
==================================

Before building any model, we must understand our data:
1. What columns do we have?
2. What data types (numbers, text)?
3. Are there missing values?
4. What's the range of values?
5. Are there any obvious patterns or issues?

This is like getting to know a new friend before working together!
"""

print("\n🔍 INITIAL DATA EXPLORATION")
print("=" * 80)

# 1. First few rows
print("\n1. FIRST 5 ROWS OF TRAINING DATA:")
print("-" * 80)
print(train_df.head())

# 2. Dataset information
print("\n2. DATASET INFORMATION:")
print("-" * 80)
print(train_df.info())

# 3. Basic statistics
print("\n3. STATISTICAL SUMMARY:")
print("-" * 80)
print(train_df.describe())

# 4. Check for missing values
print("\n4. MISSING VALUES CHECK:")
print("-" * 80)
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

print("Training Data Missing Values:")
print(missing_train[missing_train > 0] if missing_train.sum() > 0 else "No missing values!")

print("\nTesting Data Missing Values:")
print(missing_test[missing_test > 0] if missing_test.sum() > 0 else "No missing values!")

# 5. Data types breakdown
print("\n5. DATA TYPES BREAKDOWN:")
print("-" * 80)
print(f"Numerical columns: {train_df.select_dtypes(include=[np.number]).columns.tolist()}")
print(f"Categorical columns: {train_df.select_dtypes(include=['object']).columns.tolist()}")

print("\n" + "=" * 80)


# ============================================================================
# PART 4: DETAILED DATA QUALITY REPORT
# ============================================================================

"""
THEORY - What is a Data Quality Report?
========================================

A Data Quality Report helps us understand:
1. COMPLETENESS: How much data is missing?
2. UNIQUENESS: How many unique values per column?
3. VALIDITY: Are values in expected ranges?
4. CONSISTENCY: Do values make sense?

Think of it as a health check-up for your data!
"""

print("\n📊 DATA QUALITY REPORT")
print("=" * 80)

def create_data_quality_report(df, df_name="Dataset"):
    """
    Creates a comprehensive data quality report for a dataframe
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe to analyze
    df_name : str
        Name of the dataset (for display)
    
    Returns:
    --------
    pandas DataFrame with quality metrics
    """
    
    quality_report = pd.DataFrame({
        'Column': df.columns,
        'Data_Type': df.dtypes.values,
        'Missing_Count': df.isnull().sum().values,
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).values,
        'Unique_Count': df.nunique().values,
        'Unique_Percentage': (df.nunique() / len(df) * 100).values,
        'Sample_Values': [df[col].dropna().head(3).tolist() for col in df.columns]
    })
    
    print(f"\n{df_name} - QUALITY REPORT:")
    print("-" * 80)
    print(quality_report.to_string(index=False))
    print(f"\nTotal Rows: {len(df)}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return quality_report

# Generate quality reports
train_quality = create_data_quality_report(train_df, "TRAINING DATA")
test_quality = create_data_quality_report(test_df, "TESTING DATA")

print("\n" + "=" * 80)


# ============================================================================
# PART 5: EXPLORATORY DATA ANALYSIS (EDA) - NUMERICAL FEATURES
# ============================================================================

"""
THEORY - Why analyze numerical features?
=========================================

Numerical features are continuous or discrete numbers (age, score, hours, etc.)

We want to understand:
1. DISTRIBUTION: How are values spread? (Normal? Skewed?)
2. OUTLIERS: Are there extreme values that don't fit?
3. RANGE: What's the minimum and maximum?
4. CENTRAL TENDENCY: What's the average/median?

VISUALIZATIONS WE'LL USE:
- Histogram: Shows frequency distribution
- Box Plot: Shows quartiles and outliers
- KDE Plot: Smooth version of histogram
"""

print("\n📈 ANALYZING NUMERICAL FEATURES")
print("=" * 80)

# Identify numerical columns (excluding ID and target)
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# Remove ID column and target column if they exist
if 'id' in numerical_cols:
    numerical_cols.remove('id')
if 'Id' in numerical_cols:
    numerical_cols.remove('Id')

# Identify target column (usually the last numerical column in train)
# You'll need to adjust this based on your actual target column name
target_col = 'score'  # Replace with your actual target column name

if target_col in numerical_cols:
    numerical_cols.remove(target_col)

print(f"Found {len(numerical_cols)} numerical features: {numerical_cols}")
print(f"Target variable: {target_col}")


def plot_numerical_analysis(df, columns, target=None, ncols=2):
    """
    Creates comprehensive visualizations for numerical columns
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe containing the data
    columns : list
        List of numerical column names to analyze
    target : str, optional
        Name of target column (for training data)
    ncols : int
        Number of columns in the subplot grid
    """
    
    if not columns:
        print("No numerical columns to plot!")
        return
    
    nrows = (len(columns) + ncols - 1) // ncols  # Calculate rows needed
    
    # 1. Distribution Plots (Histogram + KDE)
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5*nrows))
    axes = axes.flatten() if len(columns) > 1 else [axes]
    
    for idx, col in enumerate(columns):
        ax = axes[idx]
        
        # Plot histogram with KDE
        df[col].hist(bins=30, alpha=0.5, ax=ax, edgecolor='black')
        df[col].plot(kind='kde', secondary_y=True, ax=ax, color='red', linewidth=2)
        
        ax.set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        ax.set_xlabel(col)
        ax.set_ylabel('Frequency')
        ax.grid(True, alpha=0.3)
        
        # Add statistics text
        stats_text = f'Mean: {df[col].mean():.2f}\nMedian: {df[col].median():.2f}\nStd: {df[col].std():.2f}'
        ax.text(0.7, 0.95, stats_text, transform=ax.transAxes, 
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Hide empty subplots
    for idx in range(len(columns), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('numerical_distributions.png', dpi=300, bbox_inches='tight')
    print("\n✓ Distribution plots saved as 'numerical_distributions.png'")
    plt.show()
    
    # 2. Box Plots (for outlier detection)
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5*nrows))
    axes = axes.flatten() if len(columns) > 1 else [axes]
    
    for idx, col in enumerate(columns):
        ax = axes[idx]
        
        # Create box plot
        box_plot = ax.boxplot(df[col].dropna(), vert=True, patch_artist=True)
        box_plot['boxes'][0].set_facecolor('lightblue')
        box_plot['boxes'][0].set_alpha(0.7)
        
        ax.set_title(f'Box Plot of {col}', fontsize=12, fontweight='bold')
        ax.set_ylabel(col)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Calculate and display outlier count
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        
        ax.text(0.5, 0.95, f'Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)', 
                transform=ax.transAxes, ha='center', va='top',
                fontsize=9, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    # Hide empty subplots
    for idx in range(len(columns), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('numerical_boxplots.png', dpi=300, bbox_inches='tight')
    print("✓ Box plots saved as 'numerical_boxplots.png'")
    plt.show()
    
    # 3. Correlation with target (if target is provided)
    if target and target in df.columns:
        correlations = []
        for col in columns:
            corr = df[col].corr(df[target])
            correlations.append({'Feature': col, 'Correlation': corr})
        
        corr_df = pd.DataFrame(correlations).sort_values('Correlation', ascending=False)
        
        plt.figure(figsize=(10, max(6, len(columns)*0.4)))
        colors = ['green' if x > 0 else 'red' for x in corr_df['Correlation']]
        plt.barh(corr_df['Feature'], corr_df['Correlation'], color=colors, alpha=0.7)
        plt.xlabel('Correlation with Target', fontsize=12)
        plt.title(f'Feature Correlation with {target}', fontsize=14, fontweight='bold')
        plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
        plt.grid(True, alpha=0.3, axis='x')
        plt.tight_layout()
        plt.savefig('feature_correlations.png', dpi=300, bbox_inches='tight')
        print("✓ Correlation plot saved as 'feature_correlations.png'")
        plt.show()
        
        print("\nCorrelation with Target:")
        print("-" * 40)
        print(corr_df.to_string(index=False))

# Run the analysis
if numerical_cols:
    plot_numerical_analysis(train_df, numerical_cols, target=target_col)
else:
    print("No numerical features found for analysis!")

print("\n" + "=" * 80)


# ============================================================================
# PART 6: EXPLORATORY DATA ANALYSIS (EDA) - CATEGORICAL FEATURES
# ============================================================================

"""
THEORY - Why analyze categorical features?
===========================================

Categorical features are non-numerical (gender, color, city, yes/no, etc.)

We want to understand:
1. CARDINALITY: How many unique categories?
2. FREQUENCY: Which categories are most/least common?
3. BALANCE: Are categories evenly distributed?
4. RELATIONSHIP: How do categories relate to target?

VISUALIZATIONS WE'LL USE:
- Count Plot: Shows frequency of each category
- Box Plot: Shows target distribution across categories
- Percentage bars: Shows relative proportions
"""

print("\n📊 ANALYZING CATEGORICAL FEATURES")
print("=" * 80)

# Identify categorical columns
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Remove ID columns if present
categorical_cols = [col for col in categorical_cols if 'id' not in col.lower()]

print(f"Found {len(categorical_cols)} categorical features: {categorical_cols}")


def plot_categorical_analysis(df, columns, target=None, ncols=2):
    """
    Creates comprehensive visualizations for categorical columns
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe containing the data
    columns : list
        List of categorical column names to analyze
    target : str, optional
        Name of target column (for training data)
    ncols : int
        Number of columns in the subplot grid
    """
    
    if not columns:
        print("No categorical columns to plot!")
        return
    
    nrows = (len(columns) + ncols - 1) // ncols
    
    # 1. Count Plots with Percentages
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5*nrows))
    axes = axes.flatten() if len(columns) > 1 else [axes]
    
    for idx, col in enumerate(columns):
        ax = axes[idx]
        
        # Calculate value counts and percentages
        value_counts = df[col].value_counts()
        percentages = df[col].value_counts(normalize=True) * 100
        
        # Create bar plot
        value_counts.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black', alpha=0.7)
        
        ax.set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        ax.set_xlabel(col)
        ax.set_ylabel('Count')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add percentage labels on bars
        for i, (count, pct) in enumerate(zip(value_counts, percentages)):
            ax.text(i, count, f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)
        
        # Rotate x-labels if too many categories
        if len(value_counts) > 5:
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        
        # Add statistics
        stats_text = f'Unique: {df[col].nunique()}\nMode: {df[col].mode()[0] if len(df[col].mode()) > 0 else "N/A"}'
        ax.text(0.7, 0.95, stats_text, transform=ax.transAxes,
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Hide empty subplots
    for idx in range(len(columns), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('categorical_distributions.png', dpi=300, bbox_inches='tight')
    print("\n✓ Categorical distribution plots saved as 'categorical_distributions.png'")
    plt.show()
    
    # 2. Relationship with Target (if target provided)
    if target and target in df.columns:
        fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5*nrows))
        axes = axes.flatten() if len(columns) > 1 else [axes]
        
        for idx, col in enumerate(columns):
            ax = axes[idx]
            
            # Create box plot showing target distribution for each category
            df.boxplot(column=target, by=col, ax=ax, patch_artist=True)
            
            ax.set_title(f'{target} by {col}', fontsize=12, fontweight='bold')
            ax.set_xlabel(col)
            ax.set_ylabel(target)
            plt.sca(ax)
            plt.xticks(rotation=45, ha='right')
            
            # Remove the automatic title from boxplot
            ax.get_figure().suptitle('')
        
        # Hide empty subplots
        for idx in range(len(columns), len(axes)):
            axes[idx].set_visible(False)
        
        plt.tight_layout()
        plt.savefig('categorical_vs_target.png', dpi=300, bbox_inches='tight')
        print("✓ Categorical vs target plots saved as 'categorical_vs_target.png'")
        plt.show()
        
        # Print detailed statistics
        print("\nCategorical Features Impact on Target:")
        print("-" * 80)
        for col in columns:
            print(f"\n{col}:")
            stats = df.groupby(col)[target].agg(['mean', 'median', 'std', 'count'])
            print(stats)

# Run the analysis
if categorical_cols:
    plot_categorical_analysis(train_df, categorical_cols, target=target_col)
else:
    print("No categorical features found for analysis!")

print("\n" + "=" * 80)


# ============================================================================
# PART 7: CORRELATION ANALYSIS
# ============================================================================

"""
THEORY - What is Correlation?
==============================

Correlation measures the relationship between two variables:
- Value ranges from -1 to +1
- +1: Perfect positive correlation (both increase together)
- -1: Perfect negative correlation (one increases, other decreases)
-  0: No correlation (no relationship)

WHY IS THIS IMPORTANT?
1. Helps identify which features are most related to target
2. Detects multicollinearity (features too similar to each other)
3. Guides feature selection

CORRELATION HEATMAP:
- Darker colors = stronger correlation
- We look for features highly correlated with target
- We avoid features highly correlated with each other (redundant)
"""

print("\n🔗 CORRELATION ANALYSIS")
print("=" * 80)

# Select only numerical columns for correlation
numerical_data = train_df.select_dtypes(include=[np.number])

# Calculate correlation matrix
correlation_matrix = numerical_data.corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of Numerical Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
print("\n✓ Correlation heatmap saved as 'correlation_heatmap.png'")
plt.show()

# Find highly correlated feature pairs (excluding target)
print("\nHighly Correlated Feature Pairs (|correlation| > 0.7):")
print("-" * 80)

high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append({
                'Feature 1': correlation_matrix.columns[i],
                'Feature 2': correlation_matrix.columns[j],
                'Correlation': correlation_matrix.iloc[i, j]
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False)
    print(high_corr_df.to_string(index=False))
else:
    print("No highly correlated feature pairs found!")

# Show correlation with target
if target_col in correlation_matrix.columns:
    print(f"\nCorrelation with Target ({target_col}):")
    print("-" * 80)
    target_corr = correlation_matrix[target_col].drop(target_col).sort_values(ascending=False)
    print(target_corr)

print("\n" + "=" * 80)


# ============================================================================
# SUMMARY AND RECOMMENDATIONS
# ============================================================================

print("\n📋 DATA ANALYSIS SUMMARY")
print("=" * 80)

print("\n1. DATASET OVERVIEW:")
print(f"   - Training samples: {len(train_df)}")
print(f"   - Testing samples: {len(test_df)}")
print(f"   - Total features: {len(train_df.columns) - 1}")  # Excluding target
print(f"   - Numerical features: {len(numerical_cols)}")
print(f"   - Categorical features: {len(categorical_cols)}")

print("\n2. DATA QUALITY:")
missing_total = train_df.isnull().sum().sum()
if missing_total > 0:
    print(f"   ⚠ Missing values detected: {missing_total} total")
    print("   → Action needed: Handle missing values before modeling")
else:
    print("   ✓ No missing values detected")

print("\n3. FEATURE INSIGHTS:")
if target_col in correlation_matrix.columns:
    top_features = correlation_matrix[target_col].drop(target_col).abs().sort_values(ascending=False).head(5)
    print("   Top 5 most correlated features with target:")
    for feat, corr in top_features.items():
        print(f"      - {feat}: {corr:.3f}")

print("\n4. NEXT STEPS:")
print("   ✓ Data exploration complete!")
print("   → Ready for: Feature Engineering")
print("   → Then: Model Training")
print("   → Finally: Prediction & Submission")

print("\n" + "=" * 80)
print("\n🎉 PART 1 COMPLETE: DATA QUALITY REPORT & EDA")
print("=" * 80)

"""
================================================================================
WHAT WE'VE LEARNED SO FAR:
================================================================================

1. ✓ How to load and inspect data
2. ✓ How to create a data quality report
3. ✓ How to analyze numerical features (distributions, outliers)
4. ✓ How to analyze categorical features (frequencies, relationships)
5. ✓ How to identify correlations between features
6. ✓ How to make decisions about which features to keep/modify

NEXT TUTORIAL PARTS WILL COVER:
- Part 2: Data Preprocessing & Feature Engineering
- Part 3: Model Selection & Training
- Part 4: Model Evaluation & Tuning
- Part 5: Making Predictions & Submission

================================================================================
"""